In [ ]:
import pandas as pd

names_data = pd.read_csv('datasets/gender_by_names/names.csv')
names_data.head()

len(names_data)

len(names_data['Name'].unique())

import random

names_data = names_data.drop_duplicates(subset=['Name'], keep=random.choice(['first', 'last']))

from sklearn import preprocessing

le = preprocessing.LabelEncoder()

names_data['Gender'] = le.fit_transform(names_data['Gender'])
names_data.head()

genders = ['Female', 'Male']

import string

all_letters = string.ascii_letters + ".,;'"

n_letters = len(all_letters)
all_letters

import torch

def name_to_tensor(name):
    name_in_tensor = torch.zeros(len(name), 1, n_letters)
    
    for i, letter in enumerate(name):
        name_in_tensor[i][0][all_letters.find(letter)] = 1
    
    return name_in_tensor

name_to_tensor('a')

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        combined = torch.cat((input,hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    
n_hidden = 128
n_genders = len(genders)

rnn = RNN(n_letters, n_hidden, output_size = n_genders)

iterations = 100000

criterion = nn.NLLLoss()

learning_rate = 0.005

def output_to_gender(output):
    
    top_n, top_index = output.topk(1)
    pred_i = top_index[0].item()
    pred = genders[pred_i]